In [1]:
import numpy as np
np.set_printoptions(threshold = np.nan, linewidth = 115)
import pickle
import keras
from keras import models, optimizers, layers, regularizers, metrics, losses
from keras.layers.advanced_activations import LeakyReLU, PReLU, ELU, ReLU, ThresholdedReLU
from keras.layers.core import Dense, Dropout, SpatialDropout2D, Activation
from keras.layers.convolutional import Conv2D, SeparableConv2D
from keras.models import model_from_json, Sequential

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config = config)

IMG_SIZE = 256

# Save Comparison model
def save_model(model_name, hist_str, model_str):

    pickle.dump(model_name.history, open('Training Histories/'+ hist_str + '.p', 'wb'))
    
    print("Saved " + hist_str + " to Training Histories folder")
    
    # serialize model to JSON
    model_name = model.to_json()
    with open("CNN Models/" + model_str + ".json", "w") as json_file:
        json_file.write(model_name)

    # serialize weights to HDF5
    model.save_weights("CNN Models/" + model_str + ".h5")
    print("Saved " + model_str + " and weights to CNN Models folder")
    
# Load model architecture and weights NOTE: must compile again
def load_model():
    model_str = str(input("Name of model to load: "))

    # load json and create model
    json_file = open('CNN Models/' + model_str + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("CNN Models/" + model_str + ".h5")
    print("Loaded " + model_str + " and weights from CNN Models folder")
    
    return loaded_model
    
# Load history object
def load_history():
    hist_str = str(input("Name of history to load: "))

    loaded_history = pickle.load(open('Training Histories/' + hist_str + '.p', 'rb'))
    
    print("Loaded " + hist_str + " from Training Histories folder")
    
    return loaded_history

Infiltration_class_weight = [{0: 1, 1: 12.9379183014078}]

Using TensorFlow backend.


In [2]:
# Load with pickle instead of processing images again
training_img_1 = pickle.load(open('1vAll_img_res_Infiltration_1st_half.p', 'rb'))
training_img_2 = pickle.load(open('1vAll_img_res_Infiltration_2nd_half.p', 'rb'))

training_img = np.append(training_img_1, training_img_2, axis = 0)

test_img = pickle.load(open('1vAll_test_img.p', 'rb'))

labels_1 = pickle.load(open('1vAll_labels_res_Infiltration_1st_half.p', 'rb'))
labels_2 = pickle.load(open('1vAll_labels_res_Infiltration_2nd_half.p', 'rb'))

training_labels = np.append(labels_1, labels_2, axis = 0)

test_labels = pickle.load(open('1vAll_test_labels.p', 'rb'))    

In [3]:
Infiltration_labels = training_labels[:, 8]
print('Labels shape: ', Infiltration_labels.shape)
print('Length of test_labels: ', len(test_labels))
print('No. of Infiltration Diagnoses: ', sum(Infiltration_labels))

Labels shape:  (46184,)
Length of test_labels:  3490
No. of Infiltration Diagnoses:  18149


In [4]:
# Metric Analysis
def _1vAll_accuracy(y_test, pred):
    
    pred = np.squeeze(pred, axis = -1)
    pred = np.round_(pred)
    pred = pred.astype(dtype = 'uint8')
    
    ft = pred == y_test
    
    accuracy = sum(ft)/len(ft)
        
    print('\t Complete Label Accuracy: ', (accuracy * 100), '%')
    
    print('Sum of Fully Correct Predictions: ', sum(ft))
    print('\t\t    Total Labels: ', len(ft))
    
    return accuracy

### One Vs. All

In [7]:
# Infiltration One Vs. All
# With Regularizers

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.005), input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.005)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.005)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.005)))
model.add(layers.Dense(1, activation='sigmoid'))

# Last model: Infiltration_1vAll_Reg_50e
#model = load_model()
#model.summary()

model.compile(optimizer = optimizers.RMSprop(lr = 1e-3), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

model_obj = model.fit(training_img, Infiltration_labels, 
                      epochs = 50, initial_epoch = 30, 
                      validation_split = (1 / 8), 
                      batch_size = 128, verbose = 1 )
                     #, class_weight = Infiltration_class_weight)

Predictions = model.predict(test_img)

Accuracy = _1vAll_accuracy(test_labels, Predictions)

history_str = 'Infiltration_1vAll_Reg_50e_history'
model_str   = 'Infiltration_1vAll_Reg_50e'
    
save_model(model_obj, history_str, model_str)

Train on 40411 samples, validate on 5773 samples
Epoch 1/30
40411/40411 [==============================] - 74s 2ms/step - loss: 5.2755 - acc: 0.5039 - val_loss: 0.6973 - val_acc: 0.6309
Epoch 2/30
40411/40411 [==============================] - 74s 2ms/step - loss: 0.6903 - acc: 0.6013 - val_loss: 0.6851 - val_acc: 0.6316
Epoch 3/30
40411/40411 [==============================] - 75s 2ms/step - loss: 0.6875 - acc: 0.6006 - val_loss: 0.6695 - val_acc: 0.6309
Epoch 4/30
40411/40411 [==============================] - 75s 2ms/step - loss: 0.6843 - acc: 0.6014 - val_loss: 0.6681 - val_acc: 0.6312
Epoch 5/30
40411/40411 [==============================] - 77s 2ms/step - loss: 0.7017 - acc: 0.6051 - val_loss: 0.6646 - val_acc: 0.6310
Epoch 6/30
40411/40411 [==============================] - 80s 2ms/step - loss: 0.6884 - acc: 0.6092 - val_loss: 0.6949 - val_acc: 0.6260
Epoch 7/30
40411/40411 [==============================] - 75s 2ms/step - loss: 0.6787 - acc: 0.6080 - val_loss: 0.6699 - val_acc:

In [6]:
# One Vs. All - Inception Model
# v3: Added layers
# v4: L2 Regularized

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='elu', 
                        input_shape=(IMG_SIZE, IMG_SIZE, 1), 
                        kernel_regularizer=regularizers.l2(0.005)))
model.add(layers.Conv2D(32, (3, 3), activation='elu', kernel_regularizer=regularizers.l2(0.005)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(SpatialDropout2D(0.375))
model.add(layers.Conv2D(64, (3, 3), activation='elu', kernel_regularizer=regularizers.l2(0.005)))
model.add(layers.Conv2D(64, (3, 3), activation='elu', kernel_regularizer=regularizers.l2(0.005)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SeparableConv2D(16, (3, 3), activation='elu', kernel_regularizer=regularizers.l2(0.005)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(Dropout(0.375))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='elu', kernel_regularizer=regularizers.l2(0.005)))
model.add(layers.Dense(1, activation='sigmoid'))

# Last model: 1vAll_Inception_v4_30e
#model = load_model()
#model.summary()

model.compile(optimizer = optimizers.RMSprop(lr = 1e-3), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

model_obj = model.fit(training_img, Infiltration_labels, 
                      epochs = 30, initial_epoch = 0, 
                      validation_split = (1 / 8), 
                      batch_size = 64, verbose = 1 )
                     #, class_weight = Infiltration_class_weight)

Predictions = model.predict(test_img)

Accuracy = _1vAll_accuracy(test_labels, Predictions)

history_str = '1vAll_Inception_v4_30e_history'
model_str   = '1vAll_Inception_v4_30e'
    
save_model(model_obj, history_str, model_str)

Train on 40411 samples, validate on 5773 samples
Epoch 1/30
40411/40411 [==============================] - 176s 4ms/step - loss: 1.0627 - acc: 0.5820 - val_loss: 0.6929 - val_acc: 0.6309
Epoch 2/30
40411/40411 [==============================] - 184s 5ms/step - loss: 0.8132 - acc: 0.5962 - val_loss: 0.6833 - val_acc: 0.6309
Epoch 3/30
40411/40411 [==============================] - 194s 5ms/step - loss: 0.7318 - acc: 0.5965 - val_loss: 0.6923 - val_acc: 0.6309
Epoch 4/30
40411/40411 [==============================] - 191s 5ms/step - loss: 0.7440 - acc: 0.5986 - val_loss: 0.7024 - val_acc: 0.6309
Epoch 5/30
40411/40411 [==============================] - 191s 5ms/step - loss: 0.7114 - acc: 0.5971 - val_loss: 0.6865 - val_acc: 0.6309
Epoch 6/30
40411/40411 [==============================] - 193s 5ms/step - loss: 0.7071 - acc: 0.5980 - val_loss: 0.7152 - val_acc: 0.6309
Epoch 7/30
40411/40411 [==============================] - 186s 5ms/step - loss: 0.6938 - acc: 0.5992 - val_loss: 0.6682 - v

In [ ]:
# One Vs. All - Inception Model
# v2: Using Class Weights

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='selu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(SpatialDropout2D(0.375))
model.add(layers.Conv2D(64, (3, 3), activation='selu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SeparableConv2D(16, (3, 3), activation='selu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(Dropout(0.375))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='selu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Last model: 1vAll_Inception_v2_30e
#model = load_model()
#model.summary()

model.compile(optimizer = optimizers.RMSprop(lr = 1e-3), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

model_obj = model.fit(training_img, Infiltration_labels, 
                      epochs = 30, initial_epoch = 0, 
                      validation_split = (1 / 8), 
                      batch_size = 64, verbose = 1 , 
                      class_weight = Infiltration_class_weight)

Predictions = model.predict(test_img)

Accuracy = _1vAll_accuracy(test_labels, Predictions)

history_str = '1vAll_Inception_v2_30e_history'
model_str   = '1vAll_Inception_v2_30e'
    
save_model(model_obj, history_str, model_str)